In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [73]:
diabetes_cleaned = diabetes.drop_duplicates()

uninfo_cols = []
for now in diabetes.columns:
    freq = diabetes[now].value_counts(normalize=True).max()
    if freq > 0.95:
        uninfo_cols.append(now)
    nunique_rat = diabetes[now].nunique() / diabetes[now].count()
    if nunique_rat > 0.95:
        uninfo_cols.append(now)
diabetes_cleaned = diabetes_cleaned.drop(uninfo_cols, axis=1)
diabetes_cleaned

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
763,5,139,64,35,140,28.6,0.411,26,0
764,1,96,122,0,0,22.4,0.207,27,0
765,10,101,86,37,0,45.6,1.136,38,1
766,0,141,0,0,0,42.4,0.205,29,1


In [74]:
def nan_function(x):
    return np.nan if x == 0 else x

diabetes_cleaned["Glucose"] = diabetes_cleaned["Glucose"].apply(nan_function)
diabetes_cleaned["BloodPressure"] = diabetes_cleaned["BloodPressure"].apply(nan_function)
diabetes_cleaned["SkinThickness"] = diabetes_cleaned["SkinThickness"].apply(nan_function)
diabetes_cleaned["Insulin"] = diabetes_cleaned["Insulin"].apply(nan_function)
diabetes_cleaned["BMI"] = diabetes_cleaned["BMI"].apply(nan_function)
diabetes_cleaned.isnull().mean().round(2).sort_values(ascending=False)


Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [78]:
thresh = diabetes_cleaned.shape[0]*0.7
diabetes_cleaned = diabetes_cleaned.dropna(axis=1, thresh=thresh)
diabetes_cleaned = diabetes_cleaned.dropna(thresh=6, axis=0)
diabetes_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


In [80]:
diabetes_cleaned['SkinThickness'] = diabetes_cleaned['SkinThickness'].fillna(diabetes_cleaned['SkinThickness'].median())
quart1, quart2 = diabetes_cleaned['SkinThickness'].quantile(0.25), diabetes_cleaned['SkinThickness'].quantile(0.75)
iqr = quart2 - quart1
lower = quart1 - 1.5 * iqr
upper = quart2 + 1.5 * iqr
x= diabetes_cleaned['SkinThickness']
outliers = diabetes_cleaned['SkinThickness'][(x < lower) | (x > upper)]
outliers.shape[0]

87

In [84]:
mu = x.mean()
sigma = x.std()
lower = mu - 3 * sigma
upper = mu + 3 * sigma
outliers = diabetes_cleaned['SkinThickness'][(x < lower) | (x > upper)]
outliers.shape[0]

4

In [87]:
x= diabetes_cleaned['DiabetesPedigreeFunction']
quart1, quart2 = x.quantile(0.25), x.quantile(0.75)
iqr = quart2 - quart1
lower = quart1 - 1.5 * iqr
upper = quart2 + 1.5 * iqr
outliers = diabetes_cleaned['DiabetesPedigreeFunction'][(x < lower) | (x > upper)]
outliers.shape[0]

29